### this script construct conterfactual research profile that countries only recieve domestic fund. 

For each country, following publications are retained:
- articles that are not funded
- articles that are exclusively funded by the country itself

The output of the script is the number of publications of countries in year and discipline

In [1]:
import pandas as pd

In [2]:
pubs_path="../../../data/nf_folder/Data/DerivedData/CleanedRawData/pub.pkl"

In [3]:
pub_filter_path="../../../data/nf_folder/Data/DerivedData/Derived/dep-FundInternational/pub_noInt_fund.csv"

In [4]:
pub_df=pd.read_pickle(pubs_path)
pub_df.head()

,id,year,dis,type,author,funder,author_distinct,funder_distinct,funded,IntCol,cofund,intfund
0,60694041,2017,9,1,"[Canada, United States]","[United States, United States]","[United States, Canada]",[United States],1,1,0,0
1,48573156,2012,89,1,[France],Not-Funded,[France],Not-Funded,0,0,0,0
2,55323671,2015,89,1,"[France, Chile]","[Chile, Chile, Chile, Chile, Chile]","[Chile, France]",[Chile],1,1,0,0
3,57633115,2016,31,1,"[Switzerland, United States]",[Switzerland],"[Switzerland, United States]",[Switzerland],1,1,0,0
4,64831413,2018,99,1,[China],"[China, China, China, China, China, China]",[China],[China],1,0,0,0


In [6]:
author_df=pub_df[['id','year','dis','author_distinct','funder_distinct']]
author_df=author_df.explode('author_distinct')

In [10]:
def assign_label(author,funders):
    label=1
    if funders=='Not-Funded':
        label=0
    elif len(funders)==1 and funders[0]==author:
        label=0
    return label
author_df['label']=author_df.apply(lambda x:assign_label(x.author_distinct,x.funder_distinct),axis=1)

In [12]:
author_df=author_df[author_df.label==0]
author_df=author_df.groupby(['author_distinct','year','dis']).size().reset_index()

In [13]:
author_df.head()

,author_distinct,year,dis,0
0,Afghanistan,2009,3,1
1,Afghanistan,2009,33,1
2,Afghanistan,2009,44,2
3,Afghanistan,2009,47,1
4,Afghanistan,2009,52,1


In [14]:
author_df.columns=['cntry','year','dis','count']
author_df.to_csv(pub_filter_path,index=False)